In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement).order_by(Measurement.date.desc()).first()
print(most_recent_date.date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_query = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(year_query)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
precipitation_scores = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_query).order_by(Measurement.date).all()
precipitation_scores

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = pd.DataFrame(precipitation_scores, columns=['date', 'prcp'])
precip_df.set_index('date', inplace=True)
precip_df.head()

In [ ]:
# Sort the dataframe by date
sorted_precip_df = precip_df.sort_index()
sorted_precip_df = sorted_precip_df.rename(columns={"prcp":"Precipitation"})
sorted_precip_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
chart = sorted_precip_df.plot(figsize=(12,8))
plt.legend(loc='best')
plt.title(f"Precipitation Analysis ({year_query} - {most_recent_date.date})")
chart.axes.get_xaxis().set_ticks([])
plt.xlabel('Date')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
sorted_precip_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
stations = session.query(Station).count()
print(f"There are {stations} stations available in this dataset.")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

for station in active_stations:
    print(f"Station {station[0]} has a total count of {station[1]}.")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = session.query(Measurement.station, func.count(Measurement.station), func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).first()

print (f"Station {most_active_station[0]} was the most active.")
print (f"The highest temperature recorded at Station {most_active_station[0]} was {most_active_station[3]} degrees (F).")
print (f"The lowest temperature recorded at Station {most_active_station[0]} was {most_active_station[2]} degrees (F).")
print (f"The average temperature recorded at Station {most_active_station[0]} was {most_active_station[4]:.1f} degrees (F).")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_obs = session.query(Measurement.tobs).filter(Measurement.station==most_active_station[0]).filter(Measurement.date >= year_query).order_by(Measurement.date.desc()).all()
temp_obs_df = pd.DataFrame(temp_obs)
temp_obs_df.columns = ["tobs"]
temp_obs_df.head()

In [ ]:
# Histogram plot
plt.hist(temp_obs_df["tobs"], bins=12)
plt.ylabel("Frequency")
plt.xlabel("Temperature (F)")
plt.legend(["tobs"], loc="center left")
plt.title(f"Temperature Observation for Station {most_active_station[0]} from ({year_query} - {most_recent_date.date}).", fontsize = 14)
plt.show()

# Close session

In [ ]:
# Close Session
session.close()